<a href="https://colab.research.google.com/github/ashpou1/data-processing-4313/blob/main/SQL%20join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ASHOK SHARMA POUDEL  
# 218380  
# DATA PROCESSING VISUALIZATION  4313 
# DR. NICHOLAS JACOB

##Learning Some SQL with BigQuery
The first bit below comes directly from Google, you'll need to do each of those to be successful in getting this document to work.

##Cloud Resource Manager to Create a Cloud Platform project
##Enable BigQuery APIs for the project.

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


I will use austin_bikeshare data set of google trends.

## TOTAL ROWS
I will use my project id and count my total rows in data frame of bikeshare trips

In [3]:
%%bigquery --project data-viz-338818
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`

,total_rows
0,1424786


$1424786$ is a big data. 

##LONGEST TRIP IN MINUTES
Lets find out longest trip in data set

In [4]:
%%bigquery --project data-viz-338818
SELECT 
  MAX(duration_minutes) as Longest_trip_in_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`

,Longest_trip_in_minutes
0,34238


longest trip in data is $34238$ minutes

##AVERAGE TIME FOR TRIP
lets compute average time for trip in data set. we will use start station name to end station name and duration as minutes. 

In [5]:
%%bigquery --project data-viz-338818
SELECT 
  start_station_name, 
  end_station_name,
  AVG(duration_minutes) as Average_time_for_trip
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name is not null
GROUP BY start_station_name, end_station_name

,start_station_name,end_station_name,Average_time_for_trip
0,Zilker Park West,Nueces @ 3rd,22.636364
1,Toomey Rd @ South Lamar,Toomey Rd @ South Lamar,48.985782
2,Waller & 6th St.,Waller & 6th St.,38.334477
3,Pease Park,Pease Park,30.497006
4,Waller & 6th St.,East 11th St. at Victory Grill,18.060000
...,...,...,...
13962,South Congress & Barton Springs at the Austin ...,21st & Speedway @PCL,29.971429
13963,South Congress & Barton Springs at the Austin ...,23rd & San Jacinto @ DKR Stadium,49.000000
13964,South Congress & Barton Springs at the Austin ...,Nueces & 26th,39.500000
13965,South Congress & Barton Springs at the Austin ...,23rd & Rio Grande,42.642857


We got as many as 13967 average trips time which starts from  $22.636364$ to $47.509804$. these trips represent aveage time of trip from same start and end points. 

# SQL Essentials

lets have a general look data table limiting data $10$

In [6]:
%%bigquery --project data-viz-338818
SELECT *
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
LIMIT 10


,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01+00:00,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00+00:00,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19
5,9900286143,24-Hour Kiosk (Austin B-cycle),924,2014-10-26 18:12:00+00:00,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,17
6,9900286171,24-Hour Kiosk (Austin B-cycle),869,2014-10-26 18:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,6
7,9900286214,Annual Membership (Austin B-cycle),24,2014-10-26 20:12:00+00:00,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,0
8,9900286540,24-Hour Kiosk (Austin B-cycle),117,2014-10-27 15:12:00+00:00,2536,Waller & 6th St.,2536,Waller & 6th St.,12
9,13575843,Walk Up,302,2017-01-29 16:42:52+00:00,3464,Pease Park,3464,Pease Park,47


# Compute the average time for a trip based on starting point


lets use start station name and time of trip to get average time.

In [7]:
%%bigquery --project data-viz-338818
SELECT 
  start_station_name, 
  AVG(duration_minutes) as Average_time_for_trip
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name is not null
GROUP BY start_station_name

,start_station_name,Average_time_for_trip
0,Zilker Park West,27.168259
1,Toomey Rd @ South Lamar,28.115798
2,State Capitol @ 14th & Colorado,30.421842
3,Waller & 6th St.,23.265843
4,Pease Park,28.648889
...,...,...
188,Republic Square @ Guadalupe & 4th St.,23.067313
189,3rd & West,19.937466
190,3rd/West,34.152547
191,Nueces & 26th,16.805353


Among $193$ starting points we have averege time of a trip from $24.52167$ to $27.168259$  And we can also know among $1424786$ (which is our total row or we can say total trips) we only have $193$ starting points. 

# Compute how many trips start at each starting point.

From our general view of data table we will select start station name and use bigquery to count total trips from each starting point.  

In [8]:
%%bigquery --project data-viz-338818
SELECT 
  start_station_name,
  COUNT(*) as total_trips
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
ORDER BY total_trips DESC

,start_station_name,total_trips
0,21st & Speedway @PCL,72799
1,Riverside @ S. Lamar,40635
2,City Hall / Lavaca & 2nd,36520
3,2nd & Congress,35307
4,Rainey St @ Cummings,34758
...,...,...
188,Marketing Event,4
189,Eeyore's 2018,2
190,Stolen,1
191,Eeyore's 2017,1


So, we got $193$ starting point and from $1$ to $72799$ from a individual single starting point.

lets see start station table

In [9]:
%%bigquery --project data-viz-338818

SELECT start_station_name, count(*) as popular_starting_point
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name

,start_station_name,popular_starting_point
0,Zilker Park West,1361
1,Toomey Rd @ South Lamar,3627
2,State Capitol @ 14th & Colorado,5265
3,Waller & 6th St.,5744
4,Pease Park,675
...,...,...
188,Republic Square @ Guadalupe & 4th St.,11038
189,3rd & West,23683
190,3rd/West,6634
191,Nueces & 26th,17673


So We have $193$ starting points. 

# Lets find out which is one is used most times or most popular

In [10]:
%%bigquery --project data-viz-338818

SELECT MAX(popular_starting_point) as most_popular_trips
FROM (
  SELECT start_station_name, count(*) as popular_starting_point
  FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
  GROUP BY start_station_name
)

,most_popular_trips
0,72799


The most popular one comes with $72799$ times used for starting point. 

# Lets find out which is one is used second most popular

In [11]:
%%bigquery --project data-viz-338818

SELECT start_station_name, count(*) as second_popular_starting_point
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY start_station_name
HAVING second_popular_starting_point!= (SELECT MAX(popular_starting_point) as max_trips
    FROM (
      SELECT start_station_name, count(*) as popular_starting_point
  FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
  GROUP BY start_station_name
    ))
ORDER BY second_popular_starting_point DESC
LIMIT 1
 


,start_station_name,second_popular_starting_point
0,Riverside @ S. Lamar,40635


So we have River @ s Lamar comes in the second popular start station point used for $40635$ times as starting point.   

# lets see for the trips which are started and ended as same point and were over an hour long 

In [12]:
%%bigquery --project data-viz-338818

SELECT start_station_name, duration_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name = end_station_name 


,start_station_name,duration_minutes
0,Toomey Rd @ South Lamar,82
1,Waller & 6th St.,76
2,Waller & 6th St.,75
3,Waller & 6th St.,74
4,Waller & 6th St.,73
...,...,...
228334,3rd/West,63
228335,3rd & West,63
228336,3rd/West,63
228337,3rd/West,63


So, we have a table for over $60$ minutes trips which might have different ending points than starting points

lets try to make same starting and ending points which are over $60$ minutes.

In [13]:
%%bigquery --project data-viz-338818

SELECT *, COUNTIF(duration_minutes>60) as total_trips_over_an_hours
FROM (
  SELECT start_station_name, duration_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name = end_station_name
)
GROUP BY duration_minutes, start_station_name
Order BY total_trips_over_an_hours DESC

,start_station_name,duration_minutes,total_trips_over_an_hours
0,Riverside @ S. Lamar,62,109
1,Riverside @ S. Lamar,61,95
2,Riverside @ S. Lamar,63,94
3,Riverside @ S. Lamar,69,90
4,Zilker Park,61,88
...,...,...,...
24822,26th/Nueces,40,0
24823,26th/Nueces,17,0
24824,26th/Nueces,53,0
24825,26th/Nueces,48,0


This is the table for total trips over 60 minutes and have same start and ending point. 

join the table and 

Examine the average trip on Austin bike share grouping by property_type column from the bikeshare_stations table. Is average time on trips starting from 'parkland' longer than from 'sidewalks'?

In [34]:
%%bigquery --project data-viz-338818

WITH station as (
  SELECT name, property_type
  FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations`
)
SELECT station.property_type as starting_station_type,
AVG (trips.duration_minutes) as average_ride_minutes
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations`
  JOIN station 
  ON trips.start_station_name = stations_name
  group by station.property_type
  order by average_ride_minutes desc



Executing query with job ID: c292699b-a9d8-4ffa-bfba-fd401bf22fa2
Query executing: 0.34s


ERROR:
 400 Unrecognized name: start_station_name at [2:10]

(job ID: c292699b-a9d8-4ffa-bfba-fd401bf22fa2)

                                        -----Query Job SQL Follows-----                                         

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:WITH station as (
   2:  SELECT start_station_name , property_type
   3:  FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations`
   4:)
   5:SELECT station.property_type as starting_station_type, AVG (trips.duration_minutes) as average_ride_minutes
   6:FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations`
   7:  JOIN station 
   8:  ON trips.start_station_name = stations_name
   9:  group by station.property_type
  10:  order by average_ride_minutes desc
    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |


In [ ]:
%%bigquery --project data-viz-338818

SELECT *
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
WHERE start_station_name = 'parkland'

In [ ]:
%%bigquery --project data-viz-338818
WITH t as(
SELECT name, property_type
FROM `bigquery-public-data.austin_bikeshare.bikeshare_stations`
)

SELECT t.property_type as starting_station_type,
AVG (trips.duration_minutes) as average_min
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips` as trips
JOIN t
ON trips.start_stations_name = t.name
GROUP BY t.property_type
ORDER BY average_min